## Text Processing Concepts

https://www.kaggle.com/snap/amazon-fine-food-reviews

1. Bag of words
2. Stemming
3. uni-gram ,bi-gram,n-gram
4. tf-idf (term frequency , inverse term frequency)
5. word2vec 
6. Avg-Word2vec and weighted word2vec

In [ ]:
ls

Features Information:

1. Id
2. ProductId - unique identifier for the product
3. UserId - unqiue identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review

## IMPORTING NECCESSARY LIBARIES

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

## Connection for SQLLite  

In [ ]:
ls

In [ ]:
con  = sqlite3.connect('database.sqlite')

In [ ]:
type(con)

We can use pandas to write sql query inorder to fetch the data

In [ ]:
final_data  = pd.read_sql_query("""
SELECT * 
FROM Reviews 
WHERE Score != 3
""",con)

In [ ]:
final_data.shape

In [ ]:
final_data.head()

In [ ]:
final_data['Text'][0:1:].values

Going to write a fucnction which will change the review into Positive and Negative

In [ ]:
def separation(x):
    if x < 3:
        return 'negative'
    return 'positive'

In [ ]:
final_data['Score']= final_data['Score'].map(separation)

In [ ]:
final_data.shape

## DATA CLEANING

In [ ]:
pd.read_sql_query("""
SELECT * FROM REVIEWS
WHERE Score !=3 
ORDER BY ProductID
""",con).head()

In [ ]:
pd.read_sql_query("""
SELECT UserId, COUNT(UserId)
FROM REVIEWS
GROUP BY UserId
HAVING COUNT(UserId)>1
""",con)

In [ ]:
pd.read_sql_query("""
SELECT UserId, Time, COUNT(*)
FROM REVIEWS
GROUP BY UserId, Time
HAVING COUNT(*)>1 
ORDER BY COUNT(*) DESC
""",con)

In [ ]:
final_data[(final_data['UserId'] == 'A3TVZM3ZIXG8YW') & (final_data['Time'] == 1291420800)]

In [ ]:
pd.read_sql_query("""
SELECT * 
FROM REVIEWS 
WHERE Score!=3 AND UserID = "AR5J8UI46CURR"
ORDER BY ProductID
""",con)

In [ ]:
# Sort the data based on the Product id ascending order
as_data = final_data.sort_values('ProductId',axis = 0 ,ascending = True)

In [ ]:
as_data.shape

In [ ]:
as_data.head()

In [ ]:
final_dr = as_data.drop_duplicates(subset = ["UserId","Time","Text"])

In [ ]:
final_dr.shape

In [ ]:
#how much percentage of data left 
(final_dr['Id'].size / as_data['Id'].size)*100

In [ ]:
final_dr[final_dr.HelpfulnessNumerator > final_dr.HelpfulnessDenominator]

In [ ]:
final_1 = final_dr[final_dr.HelpfulnessNumerator <= final_dr.HelpfulnessDenominator]

In [ ]:
print(final_dr.shape,final_1.shape)

In [ ]:
final_1['Score'].value_counts()

## Text processing Concepts

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

### list of stopwords

"this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college"

In [91]:
sample1 = final_1['Text'].values[0]
sample2 = final_1['Text'].values[1000]
sample3 = final_1['Text'].values[1500]
sample4 = final_1['Text'].values[4900]

In [86]:
print(sample1)
print('-----------')
print(sample2)
print('-----------')
print(sample3)
print('-----------')
print(sample4)

this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college
-----------
I was really looking forward to these pods based on the reviews.  Starbucks is good, but I prefer bolder taste.... imagine my surprise when I ordered 2 boxes - both were expired! One expired back in 2005 for gosh sakes.  I admit that Amazon agreed to credit me for cost plus part of shipping, but geez, 2 years expired!!!  I'm hoping to find local San Diego area shoppe that carries pods so that I can try something different than starbucks.
-----------
Great ingredients although, chicken should have been 1st rather than chicken broth, the only thing I do not think belongs in it is Canola oil. Canola or rapes

### Beautiful soup is best library for text processing

In [92]:
#!/usr/bin/env python3
from bs4 import BeautifulSoup

text = '''
<td><a href="http://www.irit.fr/SC">Signal et Communication</a>
<br/><a href="http://www.irit.fr/IRT">Ingénierie Réseaux et Télécommunications</a>
</td>
'''

soup = BeautifulSoup(text)

print(soup.get_text())

Signal et Communication
Ingénierie Réseaux et Télécommunications




In [96]:
print(soup.get_text())

Signal et Communication
Ingénierie Réseaux et Télécommunications




In [97]:
print("Beforeconverting\n",'-----','\n',sample4,'\n')
sp = BeautifulSoup(sample4)
print("After usingbeautifulsoup\n",'-------','\n',sp.get_text())

Beforeconverting
 ----- 
 Can't do sugar.  Have tried scores of SF Syrups.  NONE of them can touch the excellence of this product.<br /><br />Thick, delicious.  Perfect.  3 ingredients: Water, Maltitol, Natural Maple Flavor.  PERIOD.  No chemicals.  No garbage.<br /><br />Have numerous friends & family members hooked on this stuff.  My husband & son, who do NOT like "sugar free" prefer this over major label regular syrup.<br /><br />I use this as my SWEETENER in baking: cheesecakes, white brownies, muffins, pumpkin pies, etc... Unbelievably delicious...<br /><br />Can you tell I like it? :) 

After usingbeautifulsoup
 ------- 
 Can't do sugar.  Have tried scores of SF Syrups.  NONE of them can touch the excellence of this product.Thick, delicious.  Perfect.  3 ingredients: Water, Maltitol, Natural Maple Flavor.  PERIOD.  No chemicals.  No garbage.Have numerous friends & family members hooked on this stuff.  My husband & son, who do NOT like "sugar free" prefer this over major label reg

There are very n varieties  of text processing techinques are there Please go through the article to have an eye on regular expression which was very handy while doing the text preprocessing techinque

I am going to more text processing in with the help of regular exprerssion

In [99]:
import re

txt = "The rain in Spain"
x = re.sub("\s", "9", txt)
print(x)

The9rain9in9Spain


In [100]:
#https://stackoverflow.com/questions/43018030/replace-apostrophe-short-words-in-python/47091370#47091370
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase




In [101]:
test = "Hey I'm Yann, how're you and how's it going ? That's interesting: I'd love to hear more about it."
print(test)
print(decontracted(test))



Hey I'm Yann, how're you and how's it going ? That's interesting: I'd love to hear more about it.
Hey I am Yann, how are you and how is it going ? That is interesting: I would love to hear more about it.


In [102]:
#https://stackoverflow.com/questions/18082130/python-regex-to-remove-all-words-which-contains-number/18082370#18082370
import re
s = "ABCD abcd AB55 55CD A55D 5555"
re.sub("\S*\d\S*", "", s).strip()



'ABCD abcd'

In [103]:
#https://stackoverflow.com/a/5843547/4084039 Removing special characters in the text
print(sample4)
s4 = re.sub('[^A-Za-z0-9]+', ' ', sample4)
print('-------')
print(s4)


Can't do sugar.  Have tried scores of SF Syrups.  NONE of them can touch the excellence of this product.<br /><br />Thick, delicious.  Perfect.  3 ingredients: Water, Maltitol, Natural Maple Flavor.  PERIOD.  No chemicals.  No garbage.<br /><br />Have numerous friends & family members hooked on this stuff.  My husband & son, who do NOT like "sugar free" prefer this over major label regular syrup.<br /><br />I use this as my SWEETENER in baking: cheesecakes, white brownies, muffins, pumpkin pies, etc... Unbelievably delicious...<br /><br />Can you tell I like it? :)
-------
Can t do sugar Have tried scores of SF Syrups NONE of them can touch the excellence of this product br br Thick delicious Perfect 3 ingredients Water Maltitol Natural Maple Flavor PERIOD No chemicals No garbage br br Have numerous friends family members hooked on this stuff My husband son who do NOT like sugar free prefer this over major label regular syrup br br I use this as my SWEETENER in baking cheesecakes white

In [104]:
#https://gist.github.com/sebleier/554280
sw= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [105]:
print(sw)

{'its', 'between', 'few', 'by', 'isn', "wasn't", 'very', 'my', 'only', 'being', 'too', "haven't", 'should', 'of', 'during', 'themselves', 'this', 'haven', 'was', 'did', "needn't", 'yourself', 'there', 'they', 'ourselves', 'some', "doesn't", 'because', 'more', 'how', 'same', "you're", 'herself', "you've", 'she', 'couldn', 'mightn', 'll', 'theirs', 'd', 'will', 't', 'most', "shouldn't", 'he', 'such', 'your', 'an', 'under', 'br', 'hasn', "you'll", 'own', 'are', 'a', 'what', "you'd", 'now', 'than', "hasn't", 'with', 'again', 'won', 'when', 'or', 'out', 'you', 're', 'but', 'ma', 'all', 'each', 'wouldn', "couldn't", "should've", 'aren', 'have', 'then', 'y', "hadn't", 'up', "she's", 'as', 'been', 'who', 'any', "mightn't", 'against', 've', 'to', 'from', 'me', "weren't", "that'll", 'her', 'o', "mustn't", 'were', 'itself', 'after', "shan't", 'our', 'himself', 'in', 'ours', 'am', 'weren', 'through', 'yours', 'yourselves', 'on', "wouldn't", 'their', 'why', 'so', 'them', 'we', 'his', "don't", "won'

In [106]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jothiramsanjeevi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [107]:
##import nltk
##nltk.download('stopwords')
from nltk.corpus import stopwords
sw1 = stopwords.words('english')
print(sw1)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [108]:
#Applying all the concepts in the single function snippet
from tqdm import tqdm
final_processed_reviews = []
for txt in tqdm(final_1['Text'].values):
    txt = re.sub(r"http\S+", "", txt)
    txt = BeautifulSoup(txt, 'lxml').get_text()
    txt = decontracted(txt)
    txt = re.sub("\S*\d\S*", "", txt).strip()
    txt = re.sub('[^A-Za-z]+', ' ', txt)
    # https://gist.github.com/sebleier/554280
    txt = ' '.join(e.lower() for e in txt.split() if e.lower() not in sw)
    final_processed_reviews.append(txt.strip())

100%|██████████| 364131/364131 [01:23<00:00, 4350.19it/s]


In [109]:
fpr = final_processed_reviews[0:5000]

In [114]:
fpr[0:10]


['witty little book makes son laugh loud recite car driving along always sing refrain learned whales india drooping roses love new words book introduces silliness classic book willing bet son still able recite memory college',
 'grew reading sendak books watching really rosie movie incorporates love son loves however miss hard cover version paperbacks seem kind flimsy takes two hands keep pages open',
 'fun way children learn months year learn poems throughout school year like handmotions invent poem',
 'great little book read aloud nice rhythm well good repetition little ones like lines chicken soup rice child gets go months year go wonderful places like bombay nile eating well know get eat kids maurice sendak version ice skating treat roses heads long time not even know came surprise came little witty book',
 'book poetry months year goes month cute little poem go along love book really fun way learn months poems creative author purpose writing book give children fun way learn months

In [113]:
final_1['Text'][0:10].values

array(["this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college",
       'I grew up reading these Sendak books, and watching the Really Rosie movie that incorporates them, and love them. My son loves them too. I do however, miss the hard cover version. The paperbacks seem kind of flimsy and it takes two hands to keep the pages open.',
       'This is a fun way for children to learn their months of the year!  We will learn all of the poems throughout the school year.  they like the handmotions which I invent for each poem.',
       'This is a great little book to read aloud- it has a nice rhythm as well as good repetition that little ones like, in the lines about "chicken 

## 1.1 Bag of words

In [115]:
cv = CountVectorizer()
cv.fit(fpr)
print("featurenames",cv.get_feature_names()[0:10])
print('-'*50)
bow_final = cv.transform(fpr)
print('Type of bow cv ='  , type(bow_final))
#From theoery we already knows bag of words will only have a unique words
print('shape of bow cv = ',bow_final.get_shape())
print('Unique words',bow_final.get_shape()[1])

featurenames ['aaaaa', 'aafco', 'ab', 'abandon', 'abandoned', 'abdomen', 'abdominal', 'aberration', 'abide', 'abilities']
--------------------------------------------------
Type of bow cv = <class 'scipy.sparse.csr.csr_matrix'>
shape of bow cv =  (5000, 16190)
Unique words 16190


In [119]:
bow_final.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## 1.2 Bi-Grams and n-Grams 

Things to remember

1.stopwords 'not' should be avoided when doing n-grams

max_df is used for removing data values that appear too frequently, also known as "corpus-specific stop words".

 For example:

max_df = 0.50 means "It ignores terms that appear in more than 50% of the documents".

max_df = 25 means "It ignores terms that appear in more than 25 documents".

The default max_df is 1.0, which means "ignore terms that appear in more than 100% of the documents". Thus the default setting does not ignore any terms.

min_df is used for removing terms that appear too infrequently.

For example:

min_df = 0.01 means "ignore terms that appear in less than 1% of the documents".

min_df = 5 means "ignore terms that appear in less than 5 documents".

The default min_df is 1, which means "ignore terms that appear in less than 1 document". Thus, the default setting does not ignore any terms.


In [120]:
cv = CountVectorizer(ngram_range=(1,2),min_df = 10)
fngrcv = cv.fit_transform(fpr)
print('Type of ngram  cv',type(fngrcv))
print('Shape of ngram  cv',fngrcv.get_shape())
print('Number of unique words ',fngrcv.get_shape()[1])

Type of ngram  cv <class 'scipy.sparse.csr.csr_matrix'>
Shape of ngram  cv (5000, 3740)
Number of unique words  3740


In [121]:
tf_idf_cv = TfidfVectorizer(ngram_range=(1,2),min_df=10)
ftfidfcv = tf_idf_cv.fit_transform(fpr)
print('Type of tf_idf_cv',type(ftfidfcv))
print('Shape of tf_idf_cv',ftfidfcv.get_shape())
print('No of unique words in unigram & Bigram',ftfidfcv.get_shape()[1])

Type of tf_idf_cv <class 'scipy.sparse.csr.csr_matrix'>
Shape of tf_idf_cv (5000, 3740)
No of unique words in unigram & Bigram 3740


## 1.3 Word2vec 

You can use the custom build google word2vec which is morefull than our building , but it's very stress for your pc or laptop If you have interest to use the pretrained model. 
Please go through the below blog to know about the model

https://cutt.ly/om5FZPs 
<br>
http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.YPu2qxOA6Et 

In [122]:
#convert the sentence into to word so that w2v can convert into dimensions
i = 0
#list of words 
word = []
for w in fpr:
    word.append(w.split())
    i = i+1


In [123]:
word

[['witty',
  'little',
  'book',
  'makes',
  'son',
  'laugh',
  'loud',
  'recite',
  'car',
  'driving',
  'along',
  'always',
  'sing',
  'refrain',
  'learned',
  'whales',
  'india',
  'drooping',
  'roses',
  'love',
  'new',
  'words',
  'book',
  'introduces',
  'silliness',
  'classic',
  'book',
  'willing',
  'bet',
  'son',
  'still',
  'able',
  'recite',
  'memory',
  'college'],
 ['grew',
  'reading',
  'sendak',
  'books',
  'watching',
  'really',
  'rosie',
  'movie',
  'incorporates',
  'love',
  'son',
  'loves',
  'however',
  'miss',
  'hard',
  'cover',
  'version',
  'paperbacks',
  'seem',
  'kind',
  'flimsy',
  'takes',
  'two',
  'hands',
  'keep',
  'pages',
  'open'],
 ['fun',
  'way',
  'children',
  'learn',
  'months',
  'year',
  'learn',
  'poems',
  'throughout',
  'school',
  'year',
  'like',
  'handmotions',
  'invent',
  'poem'],
 ['great',
  'little',
  'book',
  'read',
  'aloud',
  'nice',
  'rhythm',
  'well',
  'good',
  'repetition',
  'l

In [124]:
w2v = Word2Vec(word,min_count=5,vector_size=100,workers=4)
print(w2v.wv.most_similar('good'))
print('-'*50)
print(w2v.wv.most_similar('worst'))

[('really', 0.9939231872558594), ('much', 0.9930710196495056), ('better', 0.9898900985717773), ('want', 0.9885678887367249), ('think', 0.9832383990287781), ('something', 0.9826959371566772), ('heat', 0.9823090434074402), ('tastes', 0.9817296862602234), ('stuff', 0.9801936745643616), ('even', 0.9796038269996643)]
--------------------------------------------------
[('major', 0.9990535974502563), ('e', 0.9989904165267944), ('course', 0.9989229440689087), ('uses', 0.9988593459129333), ('possible', 0.9988506436347961), ('huge', 0.9988475441932678), ('including', 0.9988381862640381), ('means', 0.9988246560096741), ('part', 0.9988219141960144), ('says', 0.9988151788711548)]


In [ ]:
len(list((w2v.wv[1])))

In [ ]:
w2vlst = list(w2v.wv.key_to_index)
print("number of words that occured minimum 5 times ",len(w2vlst))
print('-'*50)
print('Sample words occured :',w2vlst[0:10])